In [2]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn import ensemble, linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder

In [3]:
df = pd.read_csv("cleaned_data.csv")
df["Date mutation"] = pd.to_datetime(df["Date mutation"], dayfirst=True)
df = df.set_index("Date mutation").sort_index()
sample = df.sample(1_000_000).sort_index()
# sample = df
display(df)
display(df.isna().sum())
display(df.dtypes)

/var/folders/fj/8pmdh2z120lbtvdmmc2407th0000gn/T/ipykernel_10127/456962181.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("cleaned_data.csv")


,No disposition,Nature mutation,Valeur fonciere,Type de voie,Code voie,Voie,Code postal,Commune,Code departement,Code commune,Section,No plan,Nombre de lots,Code type local,Type local,Surface reelle bati,Nombre pieces principales,Nature culture,Surface terrain
Date mutation,,,,,,,,,,,,,,,,,,,
2018-01-02,1,Vente,1250000.0,RUE,Voie,DU DAUPHINE,26540.0,MOURS-SAINT-EUSEBE,26,218,AE,381,0,4.0,Local industriel. commercial ou assimilé,1105.0,0.0,T,3106.0
2018-01-02,1,Vente,127200.0,RUE,Voie,JEAN BAPTISTE LEBAS,59950.0,AUBY,59,28,A,2836,0,1.0,Maison,89.0,4.0,S,477.0
2018-01-02,1,Vente,121280.0,RUE,Voie,JULES FERRY,59450.0,SIN LE NOBLE,59,569,BC,516,0,1.0,Maison,74.0,4.0,S,176.0
2018-01-02,1,Vente,85000.0,RUE,Voie,DU ROUCHERET,25330.0,AMANCEY,25,15,AB,265,0,1.0,Maison,86.0,5.0,J,1150.0
2018-01-02,1,Vente,85000.0,RUE,Voie,DU ROUCHERET,25330.0,AMANCEY,25,15,AB,265,0,1.0,Maison,86.0,5.0,S,227.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-31,1,Vente,170000.0,RUE,Voie,DE MONTCORNU,45130.0,EPIEDS-EN-BEAUCE,45,134,ZS,97,0,1.0,Maison,88.0,4.0,S,800.0
2022-12-31,1,Vente,98400.0,RUE,Voie,DE RENNES,35230.0,ORGERES,35,208,E,561,0,1.0,Maison,171.0,8.0,S,826.0
2022-12-31,1,Vente,115000.0,IMP,Voie,DU MOULIN,27160.0,SAINTE-MARIE-D'ATTEZ,27,578,ZI,88,0,1.0,Maison,128.0,5.0,AG,1369.0


No disposition               0
Nature mutation              0
Valeur fonciere              0
Type de voie                 0
Code voie                    0
Voie                         0
Code postal                  0
Commune                      0
Code departement             0
Code commune                 0
Section                      0
No plan                      0
Nombre de lots               0
Code type local              0
Type local                   0
Surface reelle bati          0
Nombre pieces principales    0
Nature culture               0
Surface terrain              0
dtype: int64

No disposition                 int64
Nature mutation               object
Valeur fonciere              float64
Type de voie                  object
Code voie                     object
Voie                          object
Code postal                  float64
Commune                       object
Code departement              object
Code commune                   int64
Section                       object
No plan                        int64
Nombre de lots                 int64
Code type local              float64
Type local                    object
Surface reelle bati          float64
Nombre pieces principales    float64
Nature culture                object
Surface terrain              float64
dtype: object

In [4]:
# Preparing the data, encoding categorical data:
sample = sample.drop(["Voie", "Commune"], axis=1)
categorical_data = ["Nature mutation", "Type de voie", "Code voie", "Code departement", "Section", "Type local", "Nature culture"]
sample[categorical_data] = sample[categorical_data].astype(str)
display(sample[categorical_data])
dummies = pd.get_dummies(sample[categorical_data])
display(dummies)
sample = sample.drop(categorical_data, axis=1)
sample = pd.concat([sample, dummies], axis=1)
display(sample)
display(sample.dtypes)

,Nature mutation,Type de voie,Code voie,Code departement,Section,Type local,Nature culture
Date mutation,,,,,,,
2018-01-02,Vente,AV,Voie,31,ER,Appartement,S
2018-01-02,Vente,RUE,Voie,94,A,Appartement,S
2018-01-02,Vente,RUE,Voie,44,AA,Maison,AG
2018-01-02,Vente,RUE,Voie,95,AB,Maison,AG
2018-01-02,Vente,RUE,Voie,22,AE,Maison,T
...,...,...,...,...,...,...,...
2022-12-31,Vente,RUE,Voie,76,AK,Dépendance,S
2022-12-31,Vente,RUE,Voie,35,AD,Appartement,S
2022-12-31,Vente,PL,Voie,43,AX,Local industriel. commercial ou assimilé,S


,Nature mutation_Adjudication,Nature mutation_Echange,Nature mutation_Expropriation,Nature mutation_Vente,Nature mutation_Vente en l'état futur d'achèvement,Nature mutation_Vente terrain à bâtir,Type de voie_ACH,Type de voie_ALL,Type de voie_ART,Type de voie_AUT,...,Nature culture_PA,Nature culture_PC,Nature culture_PE,Nature culture_PH,Nature culture_PP,Nature culture_S,Nature culture_T,Nature culture_TP,Nature culture_VE,Nature culture_VI
Date mutation,,,,,,,,,,,,,,,,,,,,,
2018-01-02,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2018-01-02,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2018-01-02,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2018-01-02,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2018-01-02,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-31,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2022-12-31,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2022-12-31,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


,No disposition,Valeur fonciere,Code postal,Code commune,No plan,Nombre de lots,Code type local,Surface reelle bati,Nombre pieces principales,Surface terrain,...,Nature culture_PA,Nature culture_PC,Nature culture_PE,Nature culture_PH,Nature culture_PP,Nature culture_S,Nature culture_T,Nature culture_TP,Nature culture_VE,Nature culture_VI
Date mutation,,,,,,,,,,,,,,,,,,,,,
2018-01-02,1,142000.0,31600.0,395,107,0,2.0,60.0,2.0,151.0,...,0,0,0,0,0,1,0,0,0,0
2018-01-02,1,2625000.0,94310.0,54,171,0,2.0,55.0,3.0,920.0,...,0,0,0,0,0,1,0,0,0,0
2018-01-02,1,150000.0,44710.0,133,99,0,1.0,75.0,5.0,795.0,...,0,0,0,0,0,0,0,0,0,0
2018-01-02,1,330070.0,95430.0,120,417,0,1.0,136.0,5.0,600.0,...,0,0,0,0,0,0,0,0,0,0
2018-01-02,1,100000.0,22200.0,223,243,0,1.0,56.0,3.0,1286.0,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-31,1,60000.0,76300.0,681,50,0,3.0,0.0,0.0,157.0,...,0,0,0,0,0,1,0,0,0,0
2022-12-31,1,260000.0,35300.0,115,654,0,2.0,63.0,3.0,184.0,...,0,0,0,0,0,1,0,0,0,0
2022-12-31,1,1200000.0,43000.0,157,237,0,4.0,676.0,0.0,265.0,...,0,0,0,0,0,1,0,0,0,0


No disposition         int64
Valeur fonciere      float64
Code postal          float64
Code commune           int64
No plan                int64
                      ...   
Nature culture_S       uint8
Nature culture_T       uint8
Nature culture_TP      uint8
Nature culture_VE      uint8
Nature culture_VI      uint8
Length: 850, dtype: object

In [4]:
# Using a XGBoost model.
sample = sample.astype(float)
sample_cols = list(sample.columns)
y = sample["Valeur fonciere"]
sample_cols.remove("Valeur fonciere")
x = sample[sample_cols]
display(y)
display(x)

# Creating test and train samples.
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)


# Training the model.
xgb_r = XGBRegressor(n_estimators = 10)
xgb_r.fit(X_train, y_train)

# Predict the using the trained model.
pred = xgb_r.predict(X_test)
  
# Error Computation
rmse = np.sqrt(mse(y_test, pred))
print("Root Mean Squared Error : % f" %(rmse))

Date mutation
2018-01-02     125000.0
2018-01-02     360000.0
2018-01-02     325000.0
2018-01-02     240000.0
2018-01-02    1250000.0
                ...    
2022-12-31     260000.0
2022-12-31     470000.0
2022-12-31     170000.0
2022-12-31     150000.0
2022-12-31     470000.0
Name: Valeur fonciere, Length: 1000000, dtype: float64

,No disposition,Code postal,Code commune,No plan,Nombre de lots,Code type local,Surface reelle bati,Nombre pieces principales,Surface terrain,Nature mutation_Adjudication,...,Nature culture_PA,Nature culture_PC,Nature culture_PE,Nature culture_PH,Nature culture_PP,Nature culture_S,Nature culture_T,Nature culture_TP,Nature culture_VE,Nature culture_VI
Date mutation,,,,,,,,,,,,,,,,,,,,,
2018-01-02,1.0,47700.0,52.0,188.0,0.0,3.0,0.0,0.0,425.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2018-01-02,1.0,60300.0,612.0,185.0,0.0,1.0,103.0,5.0,710.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2018-01-02,1.0,56260.0,107.0,1073.0,0.0,1.0,140.0,7.0,389.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2018-01-02,1.0,45190.0,28.0,695.0,0.0,4.0,401.0,0.0,286.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2018-01-02,1.0,26540.0,218.0,381.0,0.0,4.0,4500.0,0.0,3107.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-31,1.0,35300.0,115.0,654.0,0.0,3.0,0.0,0.0,184.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2022-12-31,1.0,28100.0,134.0,334.0,0.0,2.0,25.0,1.0,42.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2022-12-31,1.0,45130.0,134.0,97.0,0.0,1.0,88.0,4.0,800.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


Root Mean Squared Error :  11167180.383055


In [ ]:
# Creating test and train samples.
# Using a XGBoost model.
sample = sample.astype(float)
sample_cols = list(sample.columns)
y = sample["Valeur fonciere"]
sample_cols.remove("Valeur fonciere")
x = sample[sample_cols]
display(y)
display(x)
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

# Create linear regression object
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(X_train, y_train)

# Make predictions using the testing set
y_pred = regr.predict(X_test)

# The mean squared error
print("Mean squared error: %.2f" % mse(y_test, y_pred))

# The coefficient of determination: 1 is perfect prediction
print("Coefficient of determination: %.2f" % r2_score(y_test, y_pred))